<a href="https://colab.research.google.com/github/rzl-ds/gu511/blob/master/006_environments_2_docker.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>

# environment management: `docker`

## why stop at `python`?

in the previous lecture, we talked about how we could manage our `python` environment (the collected behavior of the `python` language as defined in particular files on our computer) by creating *virtual* environments with the `anaconda` environment management tool. this is something I do on *every* project for which I write `python` code.

`python` didn't *invent* environment management, though -- similar arrangements have been around for a long time in a log of different languages.

`java`, for example, runs in one large virtual environment (the "java virtual machine", or `jvm` for short) that can (must!) be installed on any computer and defines the `java` environment on that computer in a way that is portable from one OS to the next.

this doesn't fix *everything*, though. there's still a lot of interdependency between the code you write and the other applications running on your computer (that large collection of installed software that makes up your application's environment). what if I have more than one programming language? what if my application includes a service (like `postgres` or `elasticsearch`)?

if a sysad updates some basic `linux` package or library that your code depends on, you may still run into problems. it would be nice if we could pin *all* of it, somehow...

at a certain point, developers of the world started asking this bigger question: if we're going to these lengths to make sure our `python` or `java` environments (our language-specific *runtime* environments) are consistent, what's to stop us from virtualizing everything our applications need?

why *not* pin the code, the runtime environment, the environment variables, the binaries in `/usr/bin`, the libraries in `/usr/lib`, the configurations in `/etc`, ...

over time, developers have coalesced to a particular tool for doing exactly that:

<div align="center"><img width="500px" src="https://www.docker.com/sites/default/files/social/docker_facebook_share.png"></div>

tangent: jails and user-level operating system virtualization were actually around loooooong before `docker`. this is a classic case of an old tool being "rediscovered" when solving a new-ish problem

## `docker`

[`docker`](https://docs.docker.com/get-started/) is software which allows you to create **`containers`**: independent virtual environments running as isolated processes on your computer.

in other words, `containers` are frozen environments containing all of the files (of any kind, not just related to `python`) and configuration variables that are required for your application to work. they are pinned to certain versions and you can repeatedly "create" them on any system which has `docker` installed regardless of what that system looks like

> Containers are an abstraction at the app layer that packages code and dependencies together. Multiple containers can run on the same machine and share the OS kernel with other containers, each running as isolated processes in user space

<div align="center"><img src="https://www.docker.com/sites/default/files/d8/2018-11/docker-containerized-appliction-blue-border_2.png" width="600px"></div>

to parse that diagram a bit, top to bottom:

we all have computers with **hardware** / **infrastructure**. the phrase **infrastructure** is used because there may be virtualized hardware (e.g. `ec2`) rather than physical hardware (our laptops)

sitting on top of that hardware is the **operating system**. we have been flippant about the word "operating system" so far, suggesting it was `mac`, `windows`, `linux`, `linux:ubuntu`, etc. here I mean the "kernel" layer, the code that manages the interaction with the hardware.

much of what you think of as the "`linux` environment" is actually the software (libraries and binaries) you use rather than the low-level operating system itself.

the binaries and libraries that you are used to working with (in `ubuntu` e.g.) are considered part of an **application**, not part of the operating system.

**`docker`** is itself a software application that runs (like all applications do) on top of the operating system.

it builds and manages **`containers`**, packaged and frozen collections of files, binaries, libraries, *etc.* which define a runnable application

note that the binaries and libraries -- all the non-`python` dependencies of your application, e.g. -- don't have to be the same as the ones installed on the computer you are using. this is one of the fundamental reasons we are going through this hassle, to allow us to start on any computer anywhere and reliably reproduce the **environment** in which we have developed and test our application.

### `docker` is **like** `conda`

`docker` is **like** `conda` in that we could use `conda` to manage multiple different `python` environments, and in those environments we had controlled, guaranteed code behavior.

we can use `docker` to create and run code in  multiple different application environments (pretty much just `linux`, but several flavors, and as many versions or slight variations as you'd like). if our application requires special `nvidia` libraries to support deep neural net calculations, we add that to the container. if that library is a different version or doesn't even exist on the host, no worries.

### `docker` is **unlike** `conda`

`docker` is also **unlike** `conda` in many ways, but for our purposes the most important is how it functions from a user's perspective.

with `conda`, we were working in a terminal and we `activate`d our environment. we were then "in" that `python` environment and we could run `python` code as needed. we could switch "in" and "out" of that environment with a command. `conda` changes your current terminal session.

`docker` is different: `docker` creates a *new, separate process* (a `container`). `docker` does not change anything about your current terminal session, or your current (host) operating system or environment.

by way of a perhaps strange analogy:

+ `conda` takes our existing `bash` process and modifies it slightly such that the way `python` commands and code are executed is different. this is like redecorating your single room apartment -- it's still the one room, but now your experience of it is different
+ `docker` creates a new process that is running on our machine but is completely different than our base experience and possibly not even reachable from our base experience. this is like building an extension on your house, or a guest house: the new rooms can be whatever you want and don't change anything about the room you started in

you can think of that `container` as being much like any other operating system, and the model of interacting with it is similar to how we interact with remote computers over `ssh`

+ to access it you may need to explicitly "log in"
+ being "inside" that container basically means having access to a terminal prompt where you may execute commands
+ while "inside" it would look and feel like an entire `ubuntu` application environment
+ from that "inside" prompt, commands you run are being executed inside the virtual environment, not the host operating system in which this all started

**<div align="center">that's a lot. PAUSE FOR ZOOM BREAK</div>**

## walkthrough

enough of the high-level discussion, though -- let's dive in and get some experience

### installing `docker`

let's start by installing `docker` on our `ec2` instances

**<div align="center">mini exercise: everyone installs `docker` on their `ec2`</div>**

full instructions here: https://docs.docker.com/install/linux/docker-ce/ubuntu/#install-docker-ce. steps:

```sh
sudo apt-get remove docker docker-engine docker.io containerd runc

sudo apt-get update

sudo apt-get install apt-transport-https gnupg-agent

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
sudo apt-key fingerprint 0EBFCD88

sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
   $(lsb_release -cs) \
   stable"

sudo apt-get update
sudo apt-get install docker-ce
```

### test that it worked

try running

```sh
# docker only runs as sudo by default
sudo docker run hello-world
```

you should get something like

```sh
Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
d1725b59e92d: Pull complete
Digest: sha256:0add3ace90ecb4adbf7777e9aacf18357296e799f81cabc9fde470971e499788
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.
```

in addition, `docker` gives us a helpful rundown of exactly what just happened:

```sh
To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/
```

### `images` and `containers`

that help message contained a lot of new words, but the two most important from `docker`s perspective are `image` and `container`. these are the fundamental "things" that `docker` cares about.

per the `docker` docs (that's fun), an **`image`** is

> an executable package that includes everything needed to run an application--the code, a runtime, libraries, environment variables, and configuration files.

under the hood, this is basically a `tar`-ball of all the files that are required for your program to run. it is as if we started up a brand new computer with *only* the low-level operating system files, installed only the libraries and packages we needed, and then created a snapshot

an `image` is a collection of the files we should use to define the environment in which an application runs, and a **`container`** is a single instance of that `image` being built and run on top of a real operating system. from the docs again:

> A container is a runtime instance of an image -- what the image becomes in memory when executed (that is, an image with state, or a user process)

the high-level plan for any `docker` application is to take an `image` which defines the applications with environment and we use it to build a `container` that **does something** (tbd), and does it in a consistent, reproducible way.

let's look at examples of those two pieces

#### example `image`

when we ran `sudo docker run hello-world`, the second thing `docker` said it did was it `pull`ed the `image` from [DockerHub](https://hub.docker.com/). a lot of the language and behavior in `docker` world copies the language of `git`.

*what do you think DockerHub is*?

much like `github` is a remote service we can all use to share `git` repositories, `DockerHub` is a remote service we can all use to share `image`s. we download those images using `pull`. try the following:

```sh
sudo docker pull ubuntu:latest
```

```
latest: Pulling from library/ubuntu
5667fdb72017: Pull complete
d83811f270d5: Pull complete
ee671aafb583: Pull complete
7fc152dfb3a6: Pull complete
Digest: sha256:b88f8848e9a1a4e4558ba7cfc4acc5879e1d0e7ac06401409062ad2627e6fb58
Status: Downloaded newer image for ubuntu:latest
docker.io/library/ubuntu:latest
```

this command pulls down various files which collectively define a single `image` called `ubuntu:latest`. the first piece (`ubuntu`) is a **repository** name, and you can find that particular repository [on DockerHub](https://hub.docker.com/r/library/ubuntu/).

as with `git`, repositories can be local (I created it on my computer) or shared (someone uploaded a local repository to `DockerHub`).

the second piece (`latest`) is a **tag**, and it acts much like a tag or commit `sha` does in `git` world. within the `ubuntu` repository there are [different tag values](https://hub.docker.com/r/library/ubuntu/tags/) (for different versions, for example).

the `latest` tag is a special tag which is defined for every repository and points to the most recently created image in that repository.

we can verify that we now have a new `ubuntu` `docker` `image` by running

```sh
sudo docker image ls
```

```
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
ubuntu              latest              2ca708c1c9cc        2 weeks ago         64.2MB
hello-world         latest              fce289e99eb9        9 months ago        1.84kB
```

#### example `container`

remember, the point of an `image` is to define the files and environment of an application, and actually running that application requires building a `container`, a running instance of that `image`.

way back when we ran `sudo docker run hello-world` we were told by `docker` that we should try running a command:

```sh
# don't run until you know what this does
sudo docker run -it ubuntu bash
```

the `docker run` command takes an `image` (identified by `repository[:tag]`, here `ubuntu`), builds a container based on that `image`, and runs *some* command.

you can pass the command you'd like to run as an argument to `docker run`, or most `images` will have a default command that runs when you provide none. here we want to run `bash` inside our `container`.

the two flags are working together to give you an interactive terminal for that `bash` process we start:

+ `-i` / `--interactive`: run in *interactive* mode (let me as a user interact with the command running inside that `container`)
+ `-t` / `--tty`: allocate a pseudo-TTY (a terminal)

**<div align="center">mini exercise: `run` an `ubuntu` `docker` `container`</div>**

check your regular `ec2` instance's OS version by printing the contents of a release file:

```sh
cat /etc/lsb-release
```

now create an `ubuntu` `docker` `image` with

```sh
sudo docker run -it ubuntu bash
```

your prompt should change. try the following commands at this new prompt:

```sh
cat /etc/lsb-release
hostname
ls -alh
exit
```

by default, `docker` does not get rid of used `container`s. you can force it to do so by providing the flag `--rm` to remove the container when it finishes, and that is somewhat common.

you can see the container you just created with either

```sh
sudo docker container ls --all
```

or

```sh
sudo docker ps --all
```

**<div align="center">mini exercise: clean up those containers</div>**

we don't need those containers any more and they *do* take up space -- let's get rid of them.

run

```sh
sudo docker container ls --all
```

to get the values of the `CONTAINER ID` or `NAMES` columns. then

```sh
sudo docker container rm [YOUR CONTAINER ID OR NAMES VALUES HERE]
```

verify they are gone by again running

```sh
sudo docker container ls --all
```

**<div align="center">mini exercise: `run` an `ubuntu` `docker` `container` that cleans up after itself</div>**

we're going to do exactly what we did above, but now add the `--rm` flag -- then we will verify that the container is removed when we're done with it.

check that no containers are left on our machine:

```sh
sudo docker container ls --all
```

then create an `ubuntu` container running the `bash` command, but with the `--rm` flag set

```sh
sudo docker run -it --rm ubuntu bash
```

do anything you want inside, but eventually `exit`. then check the container list again:

```sh
sudo docker container ls --all
```

### summary

with the above tools we already know how we could take any shared `docker` `image` out on `DockerHub` and create a local container:

1. get the `repository` and possibly `tag` name we want
    1. example: [the tensorflow docker image](https://hub.docker.com/r/tensorflow/tensorflow/) has `repository = tensorflow/tensorflow` and many `tag` values, including `1.11.0-gpu-py3`
1. use that info to run `sudo docker image pull repository[:tag]`
1. use that downloaded `image` to create a `container` with `sudo docker run repository[:tag]`
    1. note: `docker run` for an `image` that doesn't exist will check dockerhub for that image by default

**<div align="center">that's a lot. PAUSE FOR ZOOM BREAK</div>**

### creating specialized `image`s

I said you could build `image`s locally, but I've only shown how we go and get these `image`s from `DockerHub`. in practice, that's important for every `docker` `image` I build -- it's the starting point of all of them, at least.

`docker` knows how to build `image`s by following a series of special commands written in a plain text file called a `Dockerfile`. this `Dockerfile` is effectively a "recipe" for how `docker` can go from an initial basic `image` to your customized `image`

the official [`docker` docs](https://docs.docker.com/get-started/part2/#define-a-container-with-dockerfile) offer an example application with `Dockerfile`, and I've modified it to reflect some of the magic we already built in `tacoworld`. I've put it on `github` [here](https://github.com/RZachLamberty/docker_tacoworld_app). let's get it on our `ec2` instances using `git`:

```sh
git clone https://github.com/RZachLamberty/docker_tacoworld_app.git
```

the app itself is very simple (so simple, in fact, it breaks several of the rules I taught you just a lecture or two ago):

```python
import os
import socket

# why not?
import pandas as pd

print(f"""
HELLO {os.getenv("NAME", "world")}
my name is {socket.gethostname()}
these are the results of the tacoworld (tm) model
----------------------

I, Zach Lamberty, want tacos more than *anyone*, especially more than Eamon!
I, Eamon Lamberty, want tacos, so it's good there are enough to share!
""")
```

running it on my local laptop via `python app.py` gives:

```
HELLO world
my name is Zachs-MacBook-Pro.local
these are the results of the tacoworld (tm) model
----------------------

I, Zach Lamberty, want tacos more than *anyone*, especially more than Eamon!
I, Eamon Lamberty, want tacos, so it's good there are enough to share!
```

one of my tacolaborators has a `windows` computer, but they want to be able to run my deep neural print model as well, so I have decided to make it portable using `docker`. I need to create a `docker` `image` to share with them, and I can do that with a `Dockerfile`

here's the `Dockerfile`:

```Dockerfile
# Use an official Python runtime as a parent image
FROM python:3.8-slim

# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --trusted-host pypi.python.org -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]
```

let's go through this one line at a time:

```Dockerfile
# Use an official Python runtime as a parent image
FROM python:3.8-slim
```

the `FROM` command tells `docker` that this `image` is built on top of a different one (here, `python:3.8-slim`). the majority of `Dockerfile`s you will encounter start like this, and the resulting `image` is like an onion -- it's built on top of a smaller `image`, which itself was built on top of a smaller `image`, which ...

this image is often called the `parent` image. there is a way to build an `image` without a `parent` (called a `base` `image`) but we won't cover that right now

```Dockerfile
# Set the working directory to /app
WORKDIR /app
```

when you are operating on the command line in your `ec2` instance you have a working directory, and commands are executed relative to that. here we do the same but "inside" our linux environment as it is being built. we use the `docker` command `WORKDIR` to do this.

here we change the working directory of the following `docker` commands to be `/app` *inside our container*.

```Dockerfile
# Copy the current directory contents into the container at /app
COPY . /app
```

the `docker` `COPY` command can be used to copy files I have on my host (here, on our `ec2` instance) into our `image` -- this is the main way you add files to an `image`, by copying them into it. here we instruct `docker` to copy our files (`Dockerfile`, `app.py`, `README.md`, and `requirements.txt`) into the `/app` directory.

now our `image` will have a file `/app/app.py`, for example.

```Dockerfile
# Install any needed packages specified in requirements.txt
RUN pip install --trusted-host pypi.python.org -r requirements.txt
```

the `RUN` command allows us to specify that part of the building of this `image` should be to run some command. that command has to be defined inside that `image` by the time we try to `RUN` it, and that is not always the case!

here, `pip` is installed for us in our `parent` `image` `python:3.8-slim`, so we are free to use it already. we install the one package listed in `requirements.txt` (`pandas`) this way.

```
# Make port 80 available to the world outside this container
EXPOSE 80
```

the `EXPOSE` command is used to **expose** resources inside the container to the outside world. here we are suggesting that it is possible for you on your base `ec2` image to "connect" something (tbd) to the running container's port 80.

if you wanted to run some service that receives requests via `http` (like `jupyter`, e.g., on port `8888`) inside your container, and then be able to access it from your `ec2` instance, you would `EXPOSE 8888`, and then figure out how to send requests to that port (tbd later)

```Dockerfile
# Define environment variable
ENV NAME World
```

`docker` allows us to set environment variables within the `image` and subsequent `containers` with the `ENV` command. here we suggest that any new `container` should have an environment variable `$NAME` equal to "`World`"

```Dockerfile
# Run app.py when the container launches
CMD ["python", "app.py"]
```

finally, the `CMD` command can be used to suggest what command should be run when a new `container` is created (assuming the user doesn't suggest one of their own). here we tell `docker` to build an `image` which is expecting to run `python app.py` (the items in the list get joined together with spaces between them) on startup

altogether, again:

```Dockerfile
# Use an official Python runtime as a parent image
FROM python:3.8-slim

# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --trusted-host pypi.python.org -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]
```

we tell `docker` to `build` the `image` from this `Dockerfile` using the `docker build` command. along the way, we name the `image` with the `-t` flag and give it a value in the `repository[:tag]` format.

let's create a new `image` named `tacoworld:docker_demo`

```sh
sudo docker build -t tacoworld:docker_demo .
```

*note*: the `.` is important!! that tells `docker` what the *context* of this `build` is: the local working directory / files it should be looking at to build the `image`.

you will see a long printout like this:

```
Sending build context to Docker daemon  80.38kB
Step 1/7 : FROM python:3.8-slim
3.8-slim: Pulling from library/python
d121f8d1c412: Pull complete 
ca572574cc82: Pull complete 
38ead5def216: Pull complete 
d66d8ce1705e: Pull complete 
922f4bd7bc61: Pull complete 
Digest: sha256:0944c626f71b2f44ed45c13761f3cb97d75566261ade2b2d34f6ce2987dacbcb
Status: Downloaded newer image for python:3.8-slim
 ---> 62297c9f4e5c
Step 2/7 : WORKDIR /app
 ---> Running in 48f585c36b3b
Removing intermediate container 48f585c36b3b
 ---> f345411c943e
Step 3/7 : COPY . /app
s ---> 3ff49ccda267
Step 4/7 : RUN pip install --trusted-host pypi.python.org -r requirements.txt
 ---> Running in e8294cc32e0e
udo docker image ls --Collecting pandas
  Downloading pandas-1.1.2-cp38-cp38-manylinux1_x86_64.whl (10.4 MB)
allCollecting numpy>=1.15.4
  Downloading numpy-1.19.2-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
Collecting pytz>=2017.2
  Downloading pytz-2020.1-py2.py3-none-any.whl (510 kB)
Collecting python-dateutil>=2.7.3
  Downloading python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
Collecting six>=1.5
  Downloading six-1.15.0-py2.py3-none-any.whl (10 kB)
Installing collected packages: numpy, pytz, six, python-dateutil, pandas

Successfully installed numpy-1.19.2 pandas-1.1.2 python-dateutil-2.8.1 pytz-2020.1 six-1.15.0
Removing intermediate container e8294cc32e0e
 ---> 2789cc540105
Step 5/7 : EXPOSE 80
 ---> Running in ce14e3c5e597
Removing intermediate container ce14e3c5e597
 ---> ca5e96e674f4
Step 6/7 : ENV NAME World
 ---> Running in 780f50f9ade5
Removing intermediate container 780f50f9ade5
 ---> 2ef01705504e
Step 7/7 : CMD ["python", "app.py"]
 ---> Running in 288edf90bb0b
Removing intermediate container 288edf90bb0b
 ---> 9afd35ac54b7
Successfully built 9afd35ac54b7
Successfully tagged tacoworld:docker_demo
```

note the presence of a bunch of lines like

```
---> f345411c943e
```

after each command. `docker` is building our desired image by stepping through the `Dockerfile` command by command. the result of each command is then **cached** as a layer -- `docker` will now remember that if we *start* with the image `python:3.8-slim` and then the first command we run is `WORKDIR /app`, it has already built something that looks like that and doesn't have to rebuild it from scratch.

check out the `image` list again:

```sh
sudo docker image ls --all
```

this list will now contain the `hello-world`, `python`, `ubuntu`, and `tacoworld` images we have explicitly `run`, as well as many `<none>` images.

these `<none>` images are the **cached** layers -- images that were built on the way to having the final `tacoworld` image. if you check the lines `---> f345411c943e` in your output, they will be the values in the `IMAGE ID` column

I should also verify that I have the ability to run this code as built. I can do that using the image name I just gave my image (`tacoworld:docker_demo`):

```sh
sudo docker run --rm tacoworld:docker_demo
```

```
HELLO World
my name is 921e4f8c18dd
these are the results of the tacoworld (tm) model
----------------------

I, Zach Lamberty, want tacos more than *anyone*, especially more than Eamon!
I, Eamon Lamberty, want tacos, so it's good there are enough to share!
```

so to recap:

1. I created a `Dockerfile` that specified how to go from a parent image to my complete, working application
1. in the `Dockerfile` directory I used `docker build` to create an `image` based on that `Dockerfile` "recipe"
1. I used `docker run` to turn that `image` I created into a working `container`

I could share this with my be-`windows`ed coworker in two ways:

1. I give them the *files* via `github` and tell them the commands to run (e.g. those in the `README.md`) to build their own local `image`
1. I could `docker push` the `image` to `dockerhub` and they could `docker pull` that image locally (no need to build)

what are the advantages and disadvantages of those two approaches? when might I explicitly need to use one or the other?

**<div align="center">PAUSE FOR ZOOM BREAK</div>**

## okay but really *WHY*

perhaps this seemed like a lot of effort to go through to get a simple `python` application that *already worked* to start working *in a different place*, and you'd be right. we have definitely gone way out of our way if our goal was to just get our application working *somewhere*.

so why did we do it?

### repeatable, reliable delivery

what we bought with that extra effort was the ability to have our application work *anywhere* `docker` is installed, reliably, repeatably. I don't need to know what versions of any software are installed on my coworker's laptop (they don't need *any* of it installed, actually), they just need to have `docker` and be able to execute `docker run tacoworld:docker_demo` to run my model.

this is **already** a great reason. across all the data science projects I've worked on, one of the most difficult portions is *delivery*: handing off something to someone else that they can actually run. `docker` greatly simplifies the delivery process (provided the client has and can use `docker`, which is becoming extremely common)

### leverage work others have already done for us

in addition to that, a large segment of the software development community is already using `docker` to deliver their projects -- this means that fairly often someone has already built a `docker` image with that-hot-new-thing-you-want-to-use-but-can't-install properly installed and configure, and all you have to know is how to run `docker pull`

for example

+ [tensorflow](https://hub.docker.com/r/tensorflow/tensorflow/)
+ [pytorch](https://hub.docker.com/r/pytorch/pytorch/)
+ [mxnet](https://hub.docker.com/r/mxnet/)
+ [jupyter](https://hub.docker.com/u/jupyter/)
+ pretty much every programming language
+ pretty much [every single modern database technology](https://learn.g2crowd.com/best-docker-containers-repository)

### scalability and management

this happens in an engineering layer that might be a little bit beyond your day to day work as a data scientist, but the way that applications and models are deployed is also increasingly dominated by `docker`. tools on all three cloud platforms (amazon `ecs`, google `kubernetes` engine, azure container or kubernetes service) help automate the process of deploying up-to-date versions of `docker` containers and scaling them up and down with demand

<div align="center"><img src="https://thinkr.fr/wp-content/uploads/back-to-the-future-docker.jpg" width="800px"></div>

# END OF LECTURE

next lecture: [`REST` apps and web scraping](007_web_scraping.ipynb)